# Visualizing Citibike Trip Data

Data source: https://www.citibikenyc.com/system-data

In [1]:
url = "http://data.trilliumtransit.com/gtfs/path-nj-us/path-nj-us.zip"

In [2]:
%%bash

mkdir gtfs
cd gtfs
wget http://data.trilliumtransit.com/gtfs/path-nj-us/path-nj-us.zip
unzip path-nj-us.zip

Archive:  path-nj-us.zip
  inflating: agency.txt              
  inflating: calendar_dates.txt      
  inflating: calendar.txt            
  inflating: fare_attributes.txt     
  inflating: fare_rules.txt          
  inflating: feed_info.txt           
  inflating: frequencies.txt         
  inflating: route_directions.txt    
  inflating: routes.txt              
  inflating: shapes.txt              
  inflating: stop_times.txt          
  inflating: stops.txt               
  inflating: timetable_stop_order-new.txt  
  inflating: timetables-new.txt      
  inflating: transfers.txt           
  inflating: trips.txt               


--2017-04-08 10:17:05--  http://data.trilliumtransit.com/gtfs/path-nj-us/path-nj-us.zip
Resolving data.trilliumtransit.com... 52.24.177.63
Connecting to data.trilliumtransit.com|52.24.177.63|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180404 (176K) [application/zip]
Saving to: ‘path-nj-us.zip’

     0K .......... .......... .......... .......... .......... 28%  216K 1s
    50K .......... .......... .......... .......... .......... 56%  228K 0s
   100K .......... .......... .......... .......... .......... 85%  416K 0s
   150K .......... .......... ......                          100%  253K=0.7s

2017-04-08 10:17:06 (261 KB/s) - ‘path-nj-us.zip’ saved [180404/180404]



In [8]:
import pandas as pd
import numpy as np
import datetime as dt

In [9]:
df = pd.read_csv("data/201501-citibike-tripdata.csv")

In [10]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1346,1/1/2015 0:01,1/1/2015 0:24,455,1 Ave & E 44 St,40.750020,-73.969053,265,Stanton St & Chrystie St,40.722293,-73.991475,18660,Subscriber,1960.0,2
1,363,1/1/2015 0:02,1/1/2015 0:08,434,9 Ave & W 18 St,40.743174,-74.003664,482,W 15 St & 7 Ave,40.739355,-73.999318,16085,Subscriber,1963.0,1
2,346,1/1/2015 0:04,1/1/2015 0:10,491,E 24 St & Park Ave S,40.740964,-73.986022,505,6 Ave & W 33 St,40.749013,-73.988484,20845,Subscriber,1974.0,1
3,182,1/1/2015 0:04,1/1/2015 0:07,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,19610,Subscriber,1969.0,1
4,969,1/1/2015 0:05,1/1/2015 0:21,474,5 Ave & E 29 St,40.745168,-73.986831,432,E 7 St & Avenue A,40.726218,-73.983799,20197,Subscriber,1977.0,1


In [34]:
# id 10 = citibike
df['type_id'] = 10

In [15]:
df.starttime = pd.to_datetime(df.starttime)

In [16]:
df.stoptime = pd.to_datetime(df.stoptime)

In [17]:
df.tripduration = df.tripduration.astypeype(int)

In [31]:
df['start_lat'] = df['start station latitude'].astype(float)
df['start_lon'] = df['start station longitude'].astype(float)
df['end_lat'] = df['end station latitude'].astype(float)
df['end_lon'] = df['end station longitude'].astype(float)

In [32]:
df.dtypes

tripduration                        int64
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                    int64
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                      int64
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                              int64
usertype                           object
birth year                        float64
gender                              int64
start_lat                         float64
stop_lon                          float64
end_lat                           float64
end_lon                           float64
start_lon                         float64
dtype: object

In [43]:
output = df[['type_id','starttime','stoptime','tripduration','start_lat','start_lon','end_lat','end_lon']]

In [44]:
output.head()

,type_id,starttime,stoptime,tripduration,start_lat,start_lon,end_lat,end_lon
0,10,2015-01-01 00:01:00,2015-01-01 00:24:00,1346,40.750020,-73.969053,40.722293,-73.991475
1,10,2015-01-01 00:02:00,2015-01-01 00:08:00,363,40.743174,-74.003664,40.739355,-73.999318
2,10,2015-01-01 00:04:00,2015-01-01 00:10:00,346,40.740964,-73.986022,40.749013,-73.988484
3,10,2015-01-01 00:04:00,2015-01-01 00:07:00,182,40.683178,-73.965964,40.688515,-73.964763
4,10,2015-01-01 00:05:00,2015-01-01 00:21:00,969,40.745168,-73.986831,40.726218,-73.983799


In [45]:
output = output[output.starttime > pd.to_datetime('2015-01-05 00:00:00')]
output = output[output.starttime < pd.to_datetime('2015-01-06 04:00:00')]

In [46]:
output.head()

,type_id,starttime,stoptime,tripduration,start_lat,start_lon,end_lat,end_lon
28951,10,2015-01-05 00:01:00,2015-01-05 00:15:00,836,40.742388,-73.997262,40.726281,-73.989780
28952,10,2015-01-05 00:01:00,2015-01-05 00:09:00,522,40.722293,-73.991475,40.714215,-73.981346
28953,10,2015-01-05 00:01:00,2015-01-05 00:10:00,520,40.746745,-74.007756,40.756604,-73.997901
28954,10,2015-01-05 00:02:00,2015-01-05 00:13:00,641,40.760203,-73.964785,40.762272,-73.987882
28955,10,2015-01-05 00:02:00,2015-01-05 00:09:00,447,40.712733,-74.004607,40.705693,-74.016777


In [47]:
output.shape

(14622, 8)

In [50]:
output.to_csv("citibike_monday_raw.csv")

In [51]:
%%bash
head citibike_monday_raw.csv

,type_id,starttime,stoptime,tripduration,start_lat,start_lon,end_lat,end_lon
28951,10,2015-01-05 00:01:00,2015-01-05 00:15:00,836,40.74238787,-73.99726235,40.7262807,-73.98978041
28952,10,2015-01-05 00:01:00,2015-01-05 00:09:00,522,40.72229346,-73.99147535,40.714215,-73.981346
28953,10,2015-01-05 00:01:00,2015-01-05 00:10:00,520,40.746745,-74.007756,40.75660359,-73.9979009
28954,10,2015-01-05 00:02:00,2015-01-05 00:13:00,641,40.76020258,-73.96478473,40.76227205,-73.98788205
28955,10,2015-01-05 00:02:00,2015-01-05 00:09:00,447,40.71273266,-74.0046073,40.70569254,-74.01677685
28956,10,2015-01-05 00:05:00,2015-01-05 00:11:00,364,40.76915505,-73.98191841,40.76669671,-73.99061728
28957,10,2015-01-05 00:05:00,2015-01-05 00:11:00,343,40.75513557,-73.98658032,40.74780373,-73.9734419
28958,10,2015-01-05 00:06:00,2015-01-05 00:26:00,1200,40.74206539,-74.00443172,40.71542197,-74.01121978
28959,10,2015-01-05 00:06:00,2015-01-05 00:16:00,578,40.72873888,-74.00748842,40.7454973,-74.00197139


In [52]:
%%bash

awk -F"," -v OFS="," '{ $1="" ; $0=substr($0,2) } 1' citibike_monday_raw.csv > citibike_monday.csv

In [53]:
%%bash

head citibike_monday.csv

type_id,starttime,stoptime,tripduration,start_lat,start_lon,end_lat,end_lon
10,2015-01-05 00:01:00,2015-01-05 00:15:00,836,40.74238787,-73.99726235,40.7262807,-73.98978041
10,2015-01-05 00:01:00,2015-01-05 00:09:00,522,40.72229346,-73.99147535,40.714215,-73.981346
10,2015-01-05 00:01:00,2015-01-05 00:10:00,520,40.746745,-74.007756,40.75660359,-73.9979009
10,2015-01-05 00:02:00,2015-01-05 00:13:00,641,40.76020258,-73.96478473,40.76227205,-73.98788205
10,2015-01-05 00:02:00,2015-01-05 00:09:00,447,40.71273266,-74.0046073,40.70569254,-74.01677685
10,2015-01-05 00:05:00,2015-01-05 00:11:00,364,40.76915505,-73.98191841,40.76669671,-73.99061728
10,2015-01-05 00:05:00,2015-01-05 00:11:00,343,40.75513557,-73.98658032,40.74780373,-73.9734419
10,2015-01-05 00:06:00,2015-01-05 00:26:00,1200,40.74206539,-74.00443172,40.71542197,-74.01121978
10,2015-01-05 00:06:00,2015-01-05 00:16:00,578,40.72873888,-74.00748842,40.7454973,-74.00197139
